In [ ]:
#Imports
import functions as functions
import numpy as np

## Import elements (steps: A., B., C. of the methodology $\mathfrak{M}()$)

In [ ]:
elements = functions.import_elements('../Data/Elements.xlsx')

## Identify hypothetical paths compatible with the well-defined information, $\mathcal{H}^{\mathcal{I}'}$ using the A* algorithm (step: D. of the methodology $\mathfrak{M}()$)

In [ ]:
paths = []
for _, elem in elements[elements["type"]=="customer"].iterrows():
    p = functions.astar(elements, elem.squeeze(), num_paths=3, D=20)
    paths.extend(p)

### Print the elements in each path

In [ ]:
for p in paths:
    print(p.get_names())
print(f"Found {len(paths)} paths")

### Matrix generation

In [ ]:
H = functions.BuildHMatrix(paths, elements)

C = functions.Subset(elements, ["customer"])
T = functions.Subset(elements, ["junction"])
R = functions.Subset(elements, ["line"])

Hc, Hr, Ht = functions.MatrixDivision(elements, H, C,R,T)

## Optimization problem (step: E. of the methodology $\mathfrak{M}()$)

In [ ]:
model, Tr, P = functions.optimization_problem(C,R,T, H,Hc,Hr,Ht)

### Results explanation

#### Matrices $\hat{\mathbf{P}}$ & $\mathbf{T}_{\mathtt{R}}$

In [ ]:
Tr_sol, P_sol, = functions.transform_matrices_in_dfs(H,T,R, Tr, P)

In [ ]:
Tr_sol #print results

In [ ]:
P_sol

#### First constrain Eq. 17b)


##### left-hand side

In [ ]:
np.sum(Hr.values,axis=1)

In [ ]:
np.sum(Hr.values,axis=1) * P_sol.values

In [ ]:
np.sum(Hr.values,axis=1) * P_sol.values * np.transpose(Ht.values)

##### rigth-hand side

In [ ]:
(Tr_sol.values @ np.transpose(Hr.values))

In [ ]:
(Tr_sol.values @ np.transpose(Hr.values)) * np.transpose(Ht.values) * P_sol.values

#### Second constraint (Eq. 17c)

In [ ]:
P_sol.values @ Hc.values

#### Third constraint (Eq. 17d)

In [ ]:
sum(Tr_sol.values)

## Diagnostic function (step: F. of the methodology $\mathfrak{M}()$)

In [ ]:
results = functions.DiagnosticFunction(H, C, P_sol, Tr_sol)

## Plot results

In [ ]:
connections = functions.find_connections(elements, Tr_sol, P_sol, paths)

In [ ]:
functions.visualize_elements(elements, connections)